In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install --upgrade scikit-learn
import sklearn
sklearn.__version__

# Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('max_columns', None)


## Reading data files

In [ ]:
data_raw = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
data_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
data_test.shape

In [ ]:
data_raw.head()

In [ ]:
# changing column name to lower case
data_raw.columns = data_raw.columns.str.lower()

## SEPARATING NAMES OF OBJECTIVE DTYPE AND NUMERICAL DTYPE FEATURES 

In [ ]:
# separating the columns based on dtype
num_col_names = list(data_raw.select_dtypes(include=np.number))
cat_col_names = list(data_raw.select_dtypes(include=np.object_))

In [ ]:
def fill_null_values(df):
    dfc = df.copy()
    dfc[cat_col_names] = df[cat_col_names].fillna('NA',axis=1)
    dfc[num_col_names] = df[num_col_names].fillna(0, axis=1)
    #dfc[datetime_cols] = df[datetime_cols].fillna(0, axis=1)
    return dfc
    
data_v1 = fill_null_values(data_raw)
data_v1.head(10)

In [ ]:
data_raw.shape == data_v1.shape

In [ ]:
# new variable -> 'bathroom_per_room'
data_v1['bathroom_per_room'] = (data_v1['fullbath']+data_v1['halfbath'])/data_v1['totrmsabvgrd']

## Observations
1. In many features there are tails - positively skewed data
   **assumption - need to standardize the data**
2. yearbuilt has many houses built in 2000 - how to deal with this?
    The model will consider this data as lower the year number the worst and 
    higher the year number the better.
3. Overallcond and overallqual may be the important factor in deciding the saleprice.
4. yearremod had missing data so it was filled with 0s MEANING IT WAS NEVER REMODELED
    **How to deal with this? - onehotencoding**
5. ~500/1400 houses do not have basement AND ~1250/1400 DO NOT HAVE SECOND OR TYPE2 BASEMENT
    **DROP OR KEEP?
    BSMTFN1, BSMTFN2**
6. Many houses do not have complete basement. bsmntsqftun
7. ~850/1400 do not have 2nd floor - MISSING VALUES WERE FILLED WITH 0s
8. LOWQUALFINSQ - almost all houses have good finished floors - 1300/1400 0s.
    **drop this**
9. 850/1400 have full bath in basement and ~none halfbath. **bsmthalfbath-drop**
10. ~700/1400 do not have fireplaces - onehotencoding
11. **drop - encolsedporch, 3ssnporch, screenporch, poolarea, miscval, mosold,yrsold, id**

In [ ]:
# from sklearn.model_selection import StratifiedShuffleSplit

# split = StratifiedShuffleSplit(n_splits = 1,
#                               test_size=0.2,
#                               random_state=42)

# for train_index, test_index in split.split(data_v1, data_v1['overallqual']):
#     strat_train_set = data_v1.loc[train_index]
#     strat_test_set = data_v1.loc[test_index]

from sklearn.model_selection import train_test_split

strat_train_set, strat_test_set = train_test_split(data_v1, test_size=0.2,
                                                  random_state=42)

In [ ]:
data_v2 = strat_train_set.copy()

In [ ]:
datetime_cols = ['yearbuilt','yearremodadd','garageyrblt'] # 'yrsold','mosold', not in here

p = ['enclosedporch', '3ssnporch', 'screenporch', 
     'poolarea','bsmtfinsf2','miscval', 'id', 
     'lowqualfinsf','bsmthalfbath','saleprice',]

# remove from num_cols
for x in p+datetime_cols:
    if x in num_col_names:
        num_col_names.remove(x)
    else:
        print(x)
        
# remove from num_cols        
# for x in datetime_cols:
#     num_col_names.remove(x)

# remove names from num_cols and add to cat_cols
for x in ['mssubclass','overallqual','overallcond',]:
    num_col_names.remove(x)
    cat_col_names.append(x)

for x in datetime_cols+['yrsold','mosold',]:
    if x in num_col_names:
        num_col_names.remove(x)
    
num_col_names.append('bathroom_per_room')

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def prepare_data(df, cat_cols, num_cols, datetime_names, get_df=False):
    """pass np array or dataframe.
    It performs OneHotEncoding, StandardScaling
    
    It returns 2d nparray, when get_df=False
    It return pd.DataFrame when get_df=True
    """
    df_c1 = df[cat_cols + num_cols + datetime_names].copy()
#     y_df = df['saleprice']
    
    column_transformer = ColumnTransformer([
        ('one_hot_e',OneHotEncoder(sparse=False), cat_cols),
        ('scaling', StandardScaler(), num_cols)
    ])
    
    prepared_data = column_transformer.fit_transform(df_c1)
    
    final_data = np.concatenate((prepared_data, df_c1[datetime_names].values), axis =1)
    
    if get_df == True:
        transformed_ft = column_transformer.get_feature_names_out()
        transformed_ft = np.concatenate((transformed_ft, datetime_names))
        df_final_data = pd.DataFrame(final_data,
                                    columns = transformed_ft, index = df_c1.index)
        return df_final_data
    else:
        return final_data

In [ ]:
test_prepared = prepare_data(strat_test_set, cat_col_names,
                             num_col_names, datetime_cols, get_df=True)

In [ ]:
train_prepared = prepare_data(strat_train_set, cat_col_names, 
                              num_col_names, datetime_cols, get_df=True)

In [ ]:
not_in_train_prepared=[]
for ft in test_prepared.columns:
    if ft not in train_prepared.columns:
        not_in_train_prepared.append(ft)
        
not_in_test_prepared= []
for ft in train_prepared.columns:
    if ft not in test_prepared.columns:
        not_in_test_prepared.append(ft)

In [ ]:
print("rows in test_prepared", test_prepared.iloc[:,0].count())
print("rows in train_prepared", train_prepared.iloc[:,0].count())

In [ ]:
zeros_for_train = np.zeros((1168,len(not_in_train_prepared)), dtype=int)
zeros_for_test = np.zeros((292, len(not_in_test_prepared)),dtype=int)

train_prepared[not_in_train_prepared] = zeros_for_train
test_prepared[not_in_test_prepared] = zeros_for_test

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# from scipy.stats import randint
# from sklearn.model_selection import RandomizedSearchCV

# forest_reg1 = RandomForestRegressor(random_state=42)

# param_distribs = {
#     'n_estimators': randint(low=1, high=200),
#     'max_features': randint(low=1, high=350),
# }

# rnd_search = RandomizedSearchCV(forest_reg1,
#                                param_distributions=param_distribs,
#                                n_iter=20,
#                                cv=10,
#                                scoring='neg_mean_squared_error',
#                                random_state=42)

# strat_train_y = strat_train_set['saleprice']
# rnd_search.fit(train_prepared, strat_train_y)

In [ ]:
# rnd_search.best_estimator_

In [ ]:
best_reg_para = RandomForestRegressor(max_features=150, n_estimators=53, random_state=42)

strat_train_y= strat_train_set['saleprice']
best_reg_para.fit(train_prepared, strat_train_y)

y_pred = best_reg_para.predict(train_prepared)

In [ ]:
y_test = strat_test_set['saleprice']

# Feature names must be in the same order as they were in fit
test_prepared = test_prepared[train_prepared.columns]

print("test data score", best_reg_para.score(test_prepared, y_test))
print("train data score", best_reg_para.score(train_prepared, strat_train_y))

In [ ]:
def fill_null_values2(df, datetime_cols=None):
    dfc = df.copy()
    dfc[cat_col_names] = df[cat_col_names].fillna('NA',axis=1)
    dfc[num_col_names] = df[num_col_names].fillna(0, axis=1)
    if datetime_cols:
        dfc[datetime_cols] = df[datetime_cols].fillna(0, axis=1)
    return dfc

In [ ]:
data_test.columns = data_test.columns.str.lower()
# new variable -> 'bathroom_per_room'
data_test['bathroom_per_room'] = (data_test['fullbath'] + data_test['halfbath']) / data_test['totrmsabvgrd']

data_test = fill_null_values2(data_test, datetime_cols)

data_test_prepared = prepare_data(data_test, cat_col_names, num_col_names, datetime_cols, get_df=True)

cols_not_in_test = []
for col in train_prepared.columns:
    if col not in data_test_prepared.columns:
        cols_not_in_test.append(col)
cols_not_in_test

In [ ]:
zeros_for_test_data = np.zeros((1459, len(cols_not_in_test)), dtype=int)

data_test_prepared[cols_not_in_test] = zeros_for_test_data

data_test_prepared = data_test_prepared[train_prepared.columns]

In [ ]:
final_predictions = best_reg_para.predict(data_test_prepared)
final_predictions = pd.Series(final_predictions)
sub = pd.concat([data_test['id'],final_predictions],
                       axis=1)
sub.columns = ['Id','SalePrice']

# submission['Id'] = submission['Id']+14

In [ ]:
sub.to_csv('submission.csv', index= False)

submission = pd.read_csv('submission.csv')
submission[:10]